#### Problema:

Se requiere construir un sistema de recomendación de libros basados en los resumenes de libros y los temas (tópicos) de los mismos.

Para tal fin, se utilizará eñ [CMU Book Summary Dataset](https://www.cs.cmu.edu/~dbamman/booksummaries.html)

## Librerias

In [6]:
import csv 
import json

import pandas as pd
import numpy as np
from collections import Counter # Para contar frecuencias

# Preprocesar texto
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Modelado de tópicos 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Visualizaciones
import pyLDAvis
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore', category = DeprecationWarning) # Al instalar pyLDAvis ocasiona un warning con ipkernel

import sklearn
for lib in [sklearn, pyLDAvis, np, pd]:
    print(lib.__name__, lib.__version__)

sklearn 1.2.1
pyLDAvis 3.4.0
numpy 1.26.4
pandas 2.2.3


In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/danielml/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
!tar -xzvf "/Users/danielml/Documents/Computational/Curso_Bourbaki/Semana_7/booksummaries.tar.gz" -C"/Users/danielml/Documents/Computational/Curso_Bourbaki/Semana_7/"

x booksummaries/
x booksummaries/README
x booksummaries/booksummaries.txt


## Lectura y exploración de datos.

In [7]:
data = []
with open("/Users/danielml/Documents/Computational/Curso_Bourbaki/Semana_7/booksummaries/booksummaries.txt", 'r') as f:
    reader = csv.reader(f, dialect='excel-tab')
    for row in reader:
        data.append(row)

In [8]:
len(data)

16559

In [10]:
title = []
author = []
genre = []
summary = []

for i in range(len(data)):
    title.append(data[i][2])
    author.append(data[i][3])
    if data[i][5] == '':
        genre.append([''])
    else:
        genre.append([j for j in json.loads(data[i][5]).values()])
    summary.append(data[i][6])

df = pd.DataFrame({'Title': title, 'Author': author,
                   'Genre': genre, 'Summary': summary})

print(df.shape)
df.head(5)

(16559, 4)


,Title,Author,Genre,Summary
0,Animal Farm,George Orwell,"[Roman à clef, Satire, Children's literature, ...","Old Major, the old boar on the Manor Farm, ca..."
1,A Clockwork Orange,Anthony Burgess,"[Science Fiction, Novella, Speculative fiction...","Alex, a teenager living in near-future Englan..."
2,The Plague,Albert Camus,"[Existentialism, Fiction, Absurdist fiction, N...",The text of The Plague is divided into five p...
3,An Enquiry Concerning Human Understanding,David Hume,[],The argument of the Enquiry proceeds by a ser...
4,A Fire Upon the Deep,Vernor Vinge,"[Hard science fiction, Science Fiction, Specul...",The novel posits that space around the Milky ...
